# 02 Silver transformation
- Load config and connections
- Locate and load bronze data
- transform to silver schema
- incremental upsert to silver
- write final silver data
- validate and check 
- consolidate into production ready output

## 1) Config + connections


In [14]:
# Imports
from pathlib import Path
import io
import json

import pandas as pd
from minio import Minio

In [15]:
# Read .env for credentials
def load_env_file(path: Path) -> dict:
    if not path.exists():
        raise FileNotFoundError(f"Missing env file: {path.resolve()}")

    env = {}
    for line in path.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        env[k.strip()] = v.strip()
    return env

# Env file location in repo
ENV_FILE = Path("../docker/.env")
# Storage endpoint and config
MINIO_ENDPOINT = "localhost:9000"
MINIO_SECURE = False

# Files locations in buckets
BRONZE_BUCKET = "incident-pipeline"
BRONZE_PREFIX = "bronze/incidents_raw/"
SILVER_BUCKET = "incident-pipeline-test"
SILVER_PREFIX = "silver/incidents"
SILVER_OBJECT = f"{SILVER_PREFIX}/incidents.parquet"

# Authenticate to file storage list buckets to confirm connection
env = load_env_file(ENV_FILE)
client = Minio(
    MINIO_ENDPOINT,
    access_key=env["MINIO_ROOT_USER"],
    secret_key=env["MINIO_ROOT_PASSWORD"],
    secure=MINIO_SECURE,
)

print("Connected buckets:", [b.name for b in client.list_buckets()])


Connected buckets: ['incident-pipeline', 'incident-pipeline-test', 'mlflow-artifacts']


## 2) Locate Bronze input


In [16]:
# Find the latest bronze run by manifest path
manifest_objects = [
    obj for obj in client.list_objects(BRONZE_BUCKET, prefix=BRONZE_PREFIX, recursive=True)
    if obj.object_name.endswith("manifest.json")
]

# If no manifest files found, raise an error
if not manifest_objects:
    raise RuntimeError("No bronze manifest files found.")

# Get the latest manifest file based on the timestamp in the file path
latest_manifest_key = max(obj.object_name for obj in manifest_objects)
latest_run_prefix = latest_manifest_key.rsplit("/", 1)[0]
bronze_run_id = latest_run_prefix.split("run_ts=")[-1]

# List all files for the latest run
page_keys = sorted(
    obj.object_name
    for obj in client.list_objects(BRONZE_BUCKET, prefix=f"{latest_run_prefix}/", recursive=True)
    if obj.object_name.endswith(".json") and "incidents_raw_page_" in obj.object_name
)

# If no files found, raise an error
if not page_keys:
    raise RuntimeError(f"No bronze page files found for run: {latest_run_prefix}")

# Print manifest, run folder and number of files found
print("Latest manifest:", latest_manifest_key)
print("Run folder:", latest_run_prefix)
print("Files:", len(page_keys))


Latest manifest: bronze/incidents_raw/run_ts=20260214T142754Z/manifest.json
Run folder: bronze/incidents_raw/run_ts=20260214T142754Z
Files: 10


## 3) Read Bronze pages


In [17]:
# Read all bronze pages and combine records
bronze_rows = []

for key in page_keys:
    resp = client.get_object(BRONZE_BUCKET, key)
    try:
        payload = json.loads(resp.read().decode("utf-8"))
    finally:
        resp.close()
        resp.release_conn()

    bronze_rows.extend(payload.get("result", []))

print("Bronze rows loaded:", len(bronze_rows))


Bronze rows loaded: 10000


## 4) Transform to Silver schema


In [18]:
# Explore Bronze data structure
df_bronze = pd.DataFrame(bronze_rows)

# Data has nested fields for certain columns, function to flatten to display_value
def _extract_display_value(value):
    if isinstance(value, dict):
        if "display_value" in value:
            return value.get("display_value")
        return value
    return value

# Identify columns that contain nested values
nested_cols = [
    c for c in df_bronze.columns
    if df_bronze[c].map(lambda v: isinstance(v, dict)).any()
]

# Flatten display_value in nested reference columns into the same column
df_bronze_flat = df_bronze.copy()
for col in nested_cols:
    df_bronze_flat[col] = df_bronze_flat[col].map(_extract_display_value)

print("Nested columns flattened:", nested_cols)
pd.set_option("display.max_columns", None)
display(df_bronze_flat.head(5))

Nested columns flattened: ['opened_by', 'caller_id', 'resolved_by', 'closed_by', 'assignment_group', 'assigned_to', 'business_service', 'cmdb_ci', 'sys_domain']


,sys_id,number,task_effective_number,sys_class_name,state,incident_state,priority,impact,urgency,severity,approval,escalation,notify,opened_at,resolved_at,closed_at,activity_due,due_date,sys_created_on,sys_updated_on,sys_created_by,sys_updated_by,opened_by,caller_id,resolved_by,closed_by,assignment_group,assigned_to,business_service,cmdb_ci,category,subcategory,contact_type,short_description,description,comments,work_notes,comments_and_work_notes,work_notes_list,parent_incident,problem_id,rfc,correlation_id,correlation_display,active,knowledge,made_sla,close_code,close_notes,calendar_duration,business_duration,calendar_stc,business_stc,sys_mod_count,reassignment_count,reopen_count,sla_due,sys_domain,sys_domain_path,watch_list,group_list,additional_assignee_list,u_system,u_system_criticality,u_initial_assignment_group,u_suggested_assignment_group,u_suggested_category,u_suggested_subcategory,u_outage_day,u_outage_system
0,d98abc2af1c9ef3b3071043ad7526b01,INC1200000,INC1200000,Incident,Closed,Closed,3 - Moderate,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Notify,30-09-2024 13:37:24,01-10-2024 09:49:07,01-10-2024 13:34:27,,,30-09-2024 13:37:43,01-10-2024 13:34:27,user.100270,user.100337,User 100018,User 100121,User 100025,User 100213,Network Ops,User 100104,Business Services - Network,VPN Service,Network,DNS,Self-service,VPN Service: latency reported by multiple users,Users report latency symptoms when using VPN S...,,,,,,,,,,false,false,true,Solved (Permanently),User confirmed resolution,,,,,1,0,0,,global,/,,,,VPN Service,High,Network Ops,Network Ops,Network,DNS,false,
1,2892b7622f1606ec6f3a6a9e4347bce6,INC1200001,INC1200001,Incident,Closed,Closed,4 - Low,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Do Not Notify,24-09-2025 13:07:08,01-10-2025 10:04:44,01-10-2025 15:13:11,,,24-09-2025 13:08:26,01-10-2025 15:13:11,user.100974,user.100890,User 100198,User 100188,User 100124,User 100058,Network Ops,User 100134,Business Services - Network,VPN Service,Network,DNS,Phone,Dns problem affecting VPN Service,Users report dns symptoms when using VPN Servi...,,,,,,,,,,false,false,false,Solved (Permanently),Resolved and verified,,,,,0,0,0,,global,/,,,,VPN Service,High,Network Ops,Network Ops,Network,DNS,false,
2,d696c58fd31737311872387ccc378f65,INC1200002,INC1200002,Incident,Closed,Closed,4 - Low,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Do Not Notify,03-12-2025 08:29:11,04-12-2025 11:40:59,04-12-2025 16:04:31,,,03-12-2025 08:29:11,04-12-2025 16:04:31,user.100096,system,User 100026,User 100013,User 100167,User 100098,Network Ops,User 100102,Business Services - Network,Core Network,Network,VPN,Self-service,Vpn problem affecting Core Network,Users report vpn symptoms when using Core Netw...,,,,,,,,,,false,false,true,Solved (Permanently),User confirmed resolution,,,,,1,0,0,,global,/,,,,Core Network,High,Network Ops,Network Ops,Network,VPN,false,
3,7f3109754339aefb8f8be0b9af3804d7,INC1200003,INC1200003,Incident,Closed,Closed,3 - Moderate,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Do Not Notify,06-08-2024 10:20:43,06-08-2024 13:09:52,06-08-2024 19:00:27,,,06-08-2024 10:23:41,06-08-2024 19:00:27,user.100996,system,User 100122,User 100150,User 100001,User 100126,Identity and User Access,User 100068,Business Services - Shared,MFA,Access,Permission request,Self-service,MFA: permission reported by multiple users,Users report permission symptoms when using MF...,,,,,,,,,,false,false,true,Cancelled,Applied fix and monitored,,,,,2,0,0,,global,/,,,,MFA,High,Identity and User Access,Identity and User Access,Access,Permission request,false,
4,3deb930f4e4959f9290e16931f90db01,INC1200004,INC1200004,Incident,Closed,Closed,4 - Low,2 - Medium,3 - Low,3 - Low,Not Requested,Normal,Notify,26-07-2024 17:56:40,30-07-2024 08:06:03,30-07-2024 08:46:21,,,26-07-2024 17:59:30,30-07-2024 08:46:21,user.100981,user.100787,User 100180,User 100088,User 100214,User 100157,End User Compute,User 100107,Business Services - Shared,Print Services,Hardware,Printing,Monitoring,Pri

In [19]:
# Build the silver layer from the newly ingested bronze data
silver_new = df_bronze_flat.copy()

In [20]:
# Required columns for upsert and type normalisation
required_cols = ["sys_id", "sys_updated_on"]

# Check for required columns before attempting an upsert
missing_required = [c for c in required_cols if c not in silver_new.columns]
if missing_required:
    raise RuntimeError(f"Missing required column(s) for silver upsert: {missing_required}")

# Datetime columns to convert to datetime type
datetime_cols = [
    "sys_updated_on",
    "opened_at",
    "resolved_at",
    "closed_at",
    "sys_created_on",
    "due_date",
    "activity_due",
]

# Convert datetime columns to datetime
for dt_col in datetime_cols:
    if dt_col in silver_new.columns:
        silver_new[dt_col] = pd.to_datetime(
            silver_new[dt_col],
            errors="coerce",
            utc=True,
            dayfirst=True,
        )

# Convert boolean columns to boolean type
if "active" in silver_new.columns:
    silver_new["active"] = (
        silver_new["active"].astype(str).str.lower().map({"true": True, "false": False})
    )

# Add metadata columns for lineage
silver_new["bronze_run_id"] = bronze_run_id
silver_new["ingested_at_utc"] = pd.Timestamp.now(tz="UTC")

# Final silver output staged for upsert into silver layer
print("New transformed rows:", len(silver_new))
print("Silver columns:", len(silver_new.columns))
display(silver_new.head(5))

New transformed rows: 10000
Silver columns: 72


,sys_id,number,task_effective_number,sys_class_name,state,incident_state,priority,impact,urgency,severity,approval,escalation,notify,opened_at,resolved_at,closed_at,activity_due,due_date,sys_created_on,sys_updated_on,sys_created_by,sys_updated_by,opened_by,caller_id,resolved_by,closed_by,assignment_group,assigned_to,business_service,cmdb_ci,category,subcategory,contact_type,short_description,description,comments,work_notes,comments_and_work_notes,work_notes_list,parent_incident,problem_id,rfc,correlation_id,correlation_display,active,knowledge,made_sla,close_code,close_notes,calendar_duration,business_duration,calendar_stc,business_stc,sys_mod_count,reassignment_count,reopen_count,sla_due,sys_domain,sys_domain_path,watch_list,group_list,additional_assignee_list,u_system,u_system_criticality,u_initial_assignment_group,u_suggested_assignment_group,u_suggested_category,u_suggested_subcategory,u_outage_day,u_outage_system,bronze_run_id,ingested_at_utc
0,d98abc2af1c9ef3b3071043ad7526b01,INC1200000,INC1200000,Incident,Closed,Closed,3 - Moderate,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Notify,2024-09-30 13:37:24+00:00,2024-10-01 09:49:07+00:00,2024-10-01 13:34:27+00:00,NaT,NaT,2024-09-30 13:37:43+00:00,2024-10-01 13:34:27+00:00,user.100270,user.100337,User 100018,User 100121,User 100025,User 100213,Network Ops,User 100104,Business Services - Network,VPN Service,Network,DNS,Self-service,VPN Service: latency reported by multiple users,Users report latency symptoms when using VPN S...,,,,,,,,,,False,false,true,Solved (Permanently),User confirmed resolution,,,,,1,0,0,,global,/,,,,VPN Service,High,Network Ops,Network Ops,Network,DNS,false,,20260214T142754Z,2026-02-16 20:31:41.306286+00:00
1,2892b7622f1606ec6f3a6a9e4347bce6,INC1200001,INC1200001,Incident,Closed,Closed,4 - Low,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Do Not Notify,2025-09-24 13:07:08+00:00,2025-10-01 10:04:44+00:00,2025-10-01 15:13:11+00:00,NaT,NaT,2025-09-24 13:08:26+00:00,2025-10-01 15:13:11+00:00,user.100974,user.100890,User 100198,User 100188,User 100124,User 100058,Network Ops,User 100134,Business Services - Network,VPN Service,Network,DNS,Phone,Dns problem affecting VPN Service,Users report dns symptoms when using VPN Servi...,,,,,,,,,,False,false,false,Solved (Permanently),Resolved and verified,,,,,0,0,0,,global,/,,,,VPN Service,High,Network Ops,Network Ops,Network,DNS,false,,20260214T142754Z,2026-02-16 20:31:41.306286+00:00
2,d696c58fd31737311872387ccc378f65,INC1200002,INC1200002,Incident,Closed,Closed,4 - Low,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Do Not Notify,2025-12-03 08:29:11+00:00,2025-12-04 11:40:59+00:00,2025-12-04 16:04:31+00:00,NaT,NaT,2025-12-03 08:29:11+00:00,2025-12-04 16:04:31+00:00,user.100096,system,User 100026,User 100013,User 100167,User 100098,Network Ops,User 100102,Business Services - Network,Core Network,Network,VPN,Self-service,Vpn problem affecting Core Network,Users report vpn symptoms when using Core Netw...,,,,,,,,,,False,false,true,Solved (Permanently),User confirmed resolution,,,,,1,0,0,,global,/,,,,Core Network,High,Network Ops,Network Ops,Network,VPN,false,,20260214T142754Z,2026-02-16 20:31:41.306286+00:00
3,7f3109754339aefb8f8be0b9af3804d7,INC1200003,INC1200003,Incident,Closed,Closed,3 - Moderate,3 - Low,3 - Low,3 - Low,Not Requested,Normal,Do Not Notify,2024-08-06 10:20:43+00:00,2024-08-06 13:09:52+00:00,2024-08-06 19:00:27+00:00,NaT,NaT,2024-08-06 10:23:41+00:00,2024-08-06 19:00:27+00:00,user.100996,system,User 100122,User 100150,User 100001,User 100126,Identity and User Access,User 100068,Business Services - Shared,MFA,Access,Permission request,Self-service,MFA: permission reported by multiple users,Users report permission symptoms when using MF...,,,,,,,,,,False,false,true,Cancelled,Applied fix and monitored,,,,,2,0,0,,global,/,,,,MFA,High,Identity and User Access,Identity and User Access,Access,Permission request,false,,20260214T142754Z,2026-02-16 20:31:41.306286+00:00
4,3deb930f4e4959f9290e16931f90db01,INC1200004,

## 5) Load existing Silver


In [21]:
# Try to read the current silver dataset from file storage
try:
    resp = client.get_object(SILVER_BUCKET, SILVER_OBJECT)
    try:
        silver_existing = pd.read_parquet(io.BytesIO(resp.read()))
    finally:
        resp.close()
        resp.release_conn()
except Exception:
    silver_existing = pd.DataFrame(columns=silver_new.columns)

print("Existing silver rows:", len(silver_existing))


Existing silver rows: 0


## 6) Incremental upsert (latest-wins)


In [22]:
# Union then keep one row per sys_id using latest sys_updated_on
combined = pd.concat([silver_existing, silver_new], ignore_index=True)
combined["_sort_ts"] = combined["sys_updated_on"].fillna(pd.Timestamp("1970-01-01", tz="UTC"))

# Combine dataset into final silver output by keeping the latest updated record for each sys id
silver_final = (
    combined
    .sort_values("_sort_ts")
    .drop_duplicates(subset=["sys_id"], keep="last")
    .drop(columns=["_sort_ts"])
    .reset_index(drop=True)
)

print("Final upserted rows:", len(silver_final))


Final upserted rows: 10000


C:\Users\JHoll\AppData\Local\Temp\ipykernel_7072\4256578399.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([silver_existing, silver_new], ignore_index=True)
C:\Users\JHoll\AppData\Local\Temp\ipykernel_7072\4256578399.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([silver_existing, silver_new], ignore_index=True)


## 7) Write Silver


In [24]:
# Overwrite one silver parquet file in storage
buffer = io.BytesIO()
silver_final.to_parquet(buffer, index=False)
buffer.seek(0)

client.put_object(
    bucket_name=SILVER_BUCKET,
    object_name=SILVER_OBJECT,
    data=buffer,
    length=buffer.getbuffer().nbytes,
    content_type="application/octet-stream",
)

print(f"Silver written to: s3://{SILVER_BUCKET}/{SILVER_OBJECT}")

Silver written to: s3://incident-pipeline-test/silver/incidents/incidents.parquet


## 8) Validation / checks


In [ ]:
# Check for duplicates
duplicate_sys_ids = int(silver_final["sys_id"].duplicated().sum())

# Latested update timestamp
watermark = silver_final["sys_updated_on"].max()

# Read the silver file back from storage
resp = client.get_object(SILVER_BUCKET, SILVER_OBJECT)
try:
    silver_readback = pd.read_parquet(io.BytesIO(resp.read()))
finally:
    resp.close()
    resp.release_conn()

# Print validation results
print("New bronze rows:", len(silver_new))
print("Existing silver rows:", len(silver_existing))
print("Final silver rows:", len(silver_final))
print("Duplicate sys_id count:", duplicate_sys_ids)
print("Max sys_updated_on:", watermark)

# Print a single row sample for manual validation
if not silver_readback.empty:
    print("Sample silver record:", silver_readback.head(1).to_dict("records")[0])


New bronze rows: 10000
Existing silver rows: 0
Final silver rows: 10000
Duplicate sys_id count: 0
Max sys_updated_on: 2026-01-02 13:34:06+00:00
Sample silver record: {'sys_id': 'ea5947653a4822a6b11248e0d580bb5e', 'number': 'INC1205252', 'task_effective_number': 'INC1205252', 'sys_class_name': 'Incident', 'state': 'Closed', 'incident_state': 'Closed', 'priority': '2 - High', 'impact': '1 - High', 'urgency': '2 - High', 'severity': '3 - Low', 'approval': 'Not Requested', 'escalation': 'Normal', 'notify': 'Do Not Notify', 'opened_at': Timestamp('2024-01-01 10:44:27+0000', tz='UTC'), 'resolved_at': Timestamp('2024-01-02 11:24:12+0000', tz='UTC'), 'closed_at': Timestamp('2024-01-02 15:30:47+0000', tz='UTC'), 'activity_due': None, 'due_date': None, 'sys_created_on': Timestamp('2024-01-01 10:45:21+0000', tz='UTC'), 'sys_updated_on': Timestamp('2024-01-02 15:30:47+0000', tz='UTC'), 'sys_created_by': 'user.100151', 'sys_updated_by': 'system', 'opened_by': 'User 100132', 'caller_id': 'User 10002